In [ ]:
!pip install konlpy
!pip install gspread
import gspread
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install tqdm
!pip install tensorflow
!pip install numpy
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = ' '  ################   수정해주세요!!  데이터가 담긴 엑셀 파일의 구글 드라이브 경로

In [ ]:
##데이터 로드
pd_data = pd.read_excel(data)

In [ ]:
## 7:3 비율로 train 데이터와 test 데이터를 나눈다. (shuffle)
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(pd_data, test_size = 0.3,shuffle = True)

In [ ]:
## 전처리
train_data['내용'] = train_data['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "") 
train_data['내용'] = train_data['내용'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['내용'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(subset = ['내용']).dropna(subset = ['e/i', 'n/s', 'f/t', 'p/j'], how = 'all')
print('전처리 후 테스트용 샘플의 개수 :',len(train_data))

test_data['내용'] = test_data['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "") 
test_data['내용'] = test_data['내용'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['내용'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(subset = ['내용']).dropna(subset = ['e/i', 'n/s', 'f/t', 'p/j'], how = 'all')
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))


stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다']

okt = Okt()

X_train = []
for sentence in tqdm(train_data['내용']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

X_test = []
for sentence in tqdm(test_data['내용']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

In [ ]:
## 단어 인덱스 형성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)
print(tokenizer.word_counts.items())

In [ ]:
## 빈도수가 3회 미만인 단어는 삭제한다.
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1
tokenizer = Tokenizer(vocab_size) # 빈도수 3 미만인 단어는 제거
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_s = X_train
X_test_s = X_test

# **E/I분류를 위한 null 값 삭제 작업**

In [ ]:
y_train_ei = np.array(train_data['e/i'])  ################   mbti의 4가지 카테고리에 따라 'e/i' 또는 'n/s' 또는 'f/t' 또는 'p/j'
y_test_ei = np.array(test_data['e/i'])   ################   mbti의 4가지 카테고리에 따라 'e/i' 또는 'n/s' 또는 'f/t' 또는 'p/j'

In [ ]:
## E/I분류를 위한 null 값 삭제 작업

drop_train1_ei = [index for index, sentence in enumerate(y_train_ei) if sentence != 0 and sentence != 1 ]
drop_test1_ei = [index for index, sentence in enumerate(y_test_ei) if sentence != 0 and sentence != 1 ]


X_train_ei = np.delete(X_train_s, drop_train1_ei, axis=0)
y_train_ei = np.delete(y_train_ei, drop_train1_ei, axis=0)

X_test_ei = np.delete(X_test_s, drop_test1_ei, axis=0)
y_test_ei = np.delete(y_test_ei, drop_test1_ei, axis=0)

y_train_ei = y_train_ei.astype(float) 
y_test_ei = y_test_ei.astype(float)

drop_train_ei = [index for index, sentence in enumerate(X_train_ei) if len(sentence) < 1]
drop_test_ei = [index for index, sentence in enumerate(X_test_ei) if len(sentence) < 1]

X_train_ei = np.delete(X_train_ei, drop_train_ei, axis=0)
y_train_ei = np.delete(y_train_ei, drop_train_ei, axis=0)

X_test_ei = np.delete(X_test_ei, drop_test_ei, axis=0)
y_test_ei = np.delete(y_test_ei, drop_test_ei, axis=0)



In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 100
below_threshold_len(max_len, X_train_ei)

# 전체 데이터의 길이는 100으로 맞춘다.
X_train_ei = pad_sequences(X_train_ei, maxlen = max_len)
X_test_ei = pad_sequences(X_test_ei, maxlen = max_len)

# **N/S분류를 위한 null 값 삭제 작업**

In [ ]:
y_train_ns = np.array(train_data['n/s']) 
y_test_ns = np.array(test_data['n/s']) 

In [ ]:
## N/S분류를 위한 null 값 삭제 작업

drop_train1_ns = [index for index, sentence in enumerate(y_train_ns) if sentence != 0 and sentence != 1 ]
drop_test1_ns = [index for index, sentence in enumerate(y_test_ns) if sentence != 0 and sentence != 1 ]


X_train_ns = np.delete(X_train_s, drop_train1_ns, axis=0)
y_train_ns = np.delete(y_train_ns, drop_train1_ns, axis=0)

X_test_ns = np.delete(X_test_s, drop_test1_ns, axis=0)
y_test_ns = np.delete(y_test_ns, drop_test1_ns, axis=0)

y_train_ns = y_train_ns.astype(float) 
y_test_ns = y_test_ns.astype(float)

drop_train_ns = [index for index, sentence in enumerate(X_train_ns) if len(sentence) < 1]
drop_test_ns = [index for index, sentence in enumerate(X_test_ns) if len(sentence) < 1]

X_train_ns = np.delete(X_train_ns, drop_train_ns, axis=0)
y_train_ns = np.delete(y_train_ns, drop_train_ns, axis=0)

X_test_ns = np.delete(X_test_ns, drop_test_ns, axis=0)
y_test_ns = np.delete(y_test_ns, drop_test_ns, axis=0)



In [ ]:
max_len = 100
below_threshold_len(max_len, X_train_ns)

# 전체 데이터의 길이는 100으로 맞춘다.
X_train_ns = pad_sequences(X_train_ns, maxlen = max_len)
X_test_ns = pad_sequences(X_test_ns, maxlen = max_len)

# **F/T분류를 위한 null 값 삭제 작업**

In [ ]:
y_train_ft = np.array(train_data['f/t'])
y_test_ft = np.array(test_data['f/t']) 

In [ ]:
## F/T분류를 위한 null 값 삭제 작업
drop_train1_ft = [index for index, sentence in enumerate(y_train_ft) if sentence != 0 and sentence != 1 ]
drop_test1_ft = [index for index, sentence in enumerate(y_test_ft) if sentence != 0 and sentence != 1 ]


X_train_ft = np.delete(X_train_s, drop_train1_ft, axis=0)
y_train_ft = np.delete(y_train_ft, drop_train1_ft, axis=0)

X_test_ft = np.delete(X_test_s, drop_test1_ft, axis=0)
y_test_ft = np.delete(y_test_ft, drop_test1_ft, axis=0)

y_train_ft = y_train_ft.astype(float) 
y_test_ft = y_test_ft.astype(float)

drop_train_ft = [index for index, sentence in enumerate(X_train_ft) if len(sentence) < 1]
drop_test_ft = [index for index, sentence in enumerate(X_test_ft) if len(sentence) < 1]

X_train_ft = np.delete(X_train_ft, drop_train_ft, axis=0)
y_train_ft = np.delete(y_train_ft, drop_train_ft, axis=0)

X_test_ft = np.delete(X_test_ft, drop_test_ft, axis=0)
y_test_ft = np.delete(y_test_ft, drop_test_ft, axis=0)



In [ ]:
max_len = 100
below_threshold_len(max_len, X_train_ft)

# 전체 데이터의 길이는 100으로 맞춘다.
X_train_ft = pad_sequences(X_train_ft, maxlen = max_len)
X_test_ft = pad_sequences(X_test_ft, maxlen = max_len)

# **P/J분류를 위한 null 값 삭제 작업**

In [ ]:
y_train_pj = np.array(train_data['p/j'])  
y_test_pj = np.array(test_data['p/j'])

In [ ]:
## P/J분류를 위한 null 값 삭제 작업

drop_train1_pj = [index for index, sentence in enumerate(y_train_pj) if sentence != 0 and sentence != 1 ]
drop_test1_pj = [index for index, sentence in enumerate(y_test_pj) if sentence != 0 and sentence != 1 ]


X_train_pj = np.delete(X_train_s, drop_train1_pj, axis=0)
y_train_pj = np.delete(y_train_pj, drop_train1_pj, axis=0)

X_test_pj = np.delete(X_test_s, drop_test1_pj, axis=0)
y_test_pj = np.delete(y_test_pj, drop_test1_pj, axis=0)

y_train_pj = y_train_pj.astype(float) 
y_test_pj = y_test_pj.astype(float)

drop_train_pj = [index for index, sentence in enumerate(X_train_pj) if len(sentence) < 1]
drop_test_pj = [index for index, sentence in enumerate(X_test_pj) if len(sentence) < 1]

X_train_pj = np.delete(X_train_pj, drop_train_pj, axis=0)
y_train_pj = np.delete(y_train_pj, drop_train_pj, axis=0)

X_test_pj = np.delete(X_test_pj, drop_test_pj, axis=0)
y_test_pj = np.delete(y_test_pj, drop_test_pj, axis=0)



In [ ]:
max_len = 100
below_threshold_len(max_len, X_train_pj)

# 전체 데이터의 길이는 100으로 맞춘다.
X_train_pj = pad_sequences(X_train_pj, maxlen = max_len)
X_test_pj = pad_sequences(X_test_pj, maxlen = max_len)

# **E/I 분류 학습, 테스트**

In [ ]:
## E/I 분류 학습, 테스트

from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
es_ei = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=4)
mc_ei = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_ei = Sequential()
model_ei.add(Embedding(vocab_size, 100))
model_ei.add(LSTM(128, kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
model_ei.add(Dropout(0.5))
model_ei.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0005)))

opt = Adam(learning_rate=5e-4)
model_ei.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
history = model_ei.fit(X_train_ei, y_train_ei, epochs=15, callbacks=[es_ei, mc_ei], batch_size=64, validation_split=0.2)
loaded_model_ei = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model_ei.evaluate(X_test_ei, y_test_ei)[1]))

# **N/S 분류 학습, 테스트**

In [ ]:
## N/S 분류 학습, 테스트

es_ns = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=4)
mc_ns = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_ns = Sequential()
model_ns.add(Embedding(vocab_size, 100))
model_ns.add(LSTM(128, kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
model_ns.add(Dropout(0.5))
model_ns.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0005)))

opt = Adam(learning_rate=5e-4)
model_ns.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

history = model_ns.fit(X_train_ns, y_train_ns, epochs=15, callbacks=[es_ns, mc_ns], batch_size=64, validation_split=0.2)
loaded_model_ns = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model_ns.evaluate(X_test_ns, y_test_ns)[1]))

# **F/T 분류 학습, 테스트**

In [ ]:
## F/T 분류 학습, 테스트

es_ft = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=4)
mc_ft = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_ft = Sequential()
model_ft.add(Embedding(vocab_size, 100))
model_ft.add(LSTM(128, kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
model_ft.add(Dropout(0.5))
model_ft.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0005)))

opt = Adam(learning_rate=5e-4)
model_ft.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

history = model_ft.fit(X_train_ft, y_train_ft, epochs=15, callbacks=[es_ft, mc_ft], batch_size=64, validation_split=0.2)
loaded_model_ft = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model_ft.evaluate(X_test_ft, y_test_ft)[1]))

# **P/J 분류 학습, 테스트**

In [ ]:
## P/J 분류 학습, 테스트

es_pj = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=4)
mc_pj = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_pj = Sequential()
model_pj.add(Embedding(vocab_size, 100))
model_pj.add(LSTM(128, kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
model_pj.add(Dropout(0.5))
model_pj.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0005)))

opt = Adam(learning_rate=5e-4)
model_pj.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

history = model_pj.fit(X_train_pj, y_train_pj, epochs=15, callbacks=[es_pj, mc_pj], batch_size=64, validation_split=0.2)
loaded_model_pj = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model_pj.evaluate(X_test_pj, y_test_pj)[1]))

In [ ]:
pred_ei = loaded_model_ei.evaluate(X_test_ei, y_test_ei)[1]
pred_ns = loaded_model_ns.evaluate(X_test_ns, y_test_ns)[1]
pred_ft = loaded_model_ft.evaluate(X_test_ft, y_test_ft)[1]
pred_pj = loaded_model_pj.evaluate(X_test_pj, y_test_pj)[1]

In [ ]:
## 각 지표의 정확도

print("\n E/I 테스트 정확도: %.4f" % pred_ei)
print("\n N/S 테스트 정확도: %.4f" % pred_ns)
print("\n F/T 테스트 정확도: %.4f" % pred_ft)
print("\n P/J 테스트 정확도: %.4f" % pred_pj)